In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key_credential = os.environ["AZURE_SEARCH_ADMIN_KEY"] if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else None
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"] if len(os.environ["AZURE_OPENAI_API_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

credential = key_credential or DefaultAzureCredential()

In [2]:
from langsmith import Client
client = Client()

os.environ['LANGCHAIN_PROJECT'] = "contoso-coffee-chain"

In [3]:
from langchain_openai import AzureOpenAIEmbeddings
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

# Use API key if provided, otherwise use RBAC authentication
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)   

In [4]:
from langchain.vectorstores.azuresearch import AzureSearch

vector_store = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="contoso-semantic-config"
)

In [5]:
retriever = vector_store.as_retriever(k=10)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel

from langchain_core.prompts import PromptTemplate



condense_question_template = """
    As a chatbot for Contoso Coffee Inc., your tasks are:
	• Menu contains coffees, teas, bakery items, sandwiches and smoothies. Guide customers through the menu and answer their inquiries about the menu items. Receive and process orders, and handle complaints with empathy.
	• Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Do not use 'their', use 'our' instead. And use "please let me know" instead of "please let our staff know". Focus solely on cafe-related queries and ordering. 
	• Keep responses brief and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
	• Inform the customer politely if an item they request is not available in the listed menu items below.
	• If a customer likes to update or cancel the order, please help accordingly.
	• Capture any additional notes the customer may have for the menu items.
	• When listing an item in the order, mention the actual price of the item in parenthesis without multiplying it with the quantity.
	• For subtotal of the order, calculate it step-by-step: 
		○ Total price for each ordered item = price of each ordered item * quantity of each ordered item (do this for all ordered items)
		○ Subtotal = sum of total prices for all ordered items
		○ Tax amount = 0.095 * Subtotal
		○ Total = Subtotal + Tax amount
	• Do not provide the total amount of the order until asked for. If and when asked, only mention the final sum in dollars without providing the entire calculation. As mentioned earlier, remember to be brief in your responses.
	• At the end of each conversation, always obtain the customer's name, even if provided in a single word, and attach it to the order.
	• Once the customer provides their name, confirm it and state that the order can be picked up at our cafe in 15 minutes.
	• Contoso accepts only pickup orders. Payment is accepted only at the store, and if a customer suggests paying over the phone, inform them that payments are accepted solely at the store.
Remember to always maintain a polite and professional tone.

Conversaion History:
{chat_history}

Question: 
{question}

Menu Items: 
{context}

Answer:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_question_template)

print(CONDENSE_QUESTION_PROMPT)

input_variables=['chat_history', 'context', 'question'] template='\n    As a chatbot for Contoso Coffee Inc., your tasks are:\n\t• Menu contains coffees, teas, bakery items, sandwiches and smoothies. Guide customers through the menu and answer their inquiries about the menu items. Receive and process orders, and handle complaints with empathy.\n\t• Since you are representing Contoso, NEVER MENTION \'I am an AI language model\'. Do not use \'their\', use \'our\' instead. And use "please let me know" instead of "please let our staff know". Focus solely on cafe-related queries and ordering. \n\t• Keep responses brief and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.\n\t• Inform the customer politely if an item they request is not available in the listed menu items below.\n\t• If a customer like

In [7]:
print(CONDENSE_QUESTION_PROMPT.template)


    As a chatbot for Contoso Coffee Inc., your tasks are:
	• Menu contains coffees, teas, bakery items, sandwiches and smoothies. Guide customers through the menu and answer their inquiries about the menu items. Receive and process orders, and handle complaints with empathy.
	• Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Do not use 'their', use 'our' instead. And use "please let me know" instead of "please let our staff know". Focus solely on cafe-related queries and ordering. 
	• Keep responses brief and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
	• Inform the customer politely if an item they request is not available in the listed menu items below.
	• If a customer likes to update or cancel the order, please help accordingly.
	• Capture any additional

In [9]:
from langchain_openai.chat_models import AzureChatOpenAI
chat = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4",
    streaming=True,
)

In [10]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    chat, retriever, contextualize_q_prompt
)

# docs = history_aware_retriever.invoke({"input":"what french coffee options do you have?"})
# for doc in docs:
#     print(doc.page_content)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """
    As a chatbot for Contoso Coffee Inc., your tasks are:
	• Menu contains coffees, teas, bakery items, sandwiches and smoothies. Guide customers through the menu and answer their inquiries about the menu items. Receive and process orders, and handle complaints with empathy.
	• Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Do not use 'their', use 'our' instead. And use "please let me know" instead of "please let our staff know". Focus solely on cafe-related queries and ordering. 
	• Keep responses brief and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
	• Inform the customer politely if an item they request is not available in the listed menu items below.
	• If a customer likes to update or cancel the order, please help accordingly.
	• Capture any additional notes the customer may have for the menu items.
	• When listing an item in the order, mention the actual price of the item in parenthesis without multiplying it with the quantity.
	• For subtotal of the order, calculate it step-by-step: 
		○ Total price for each ordered item = price of each ordered item * quantity of each ordered item (do this for all ordered items)
		○ Subtotal = sum of total prices for all ordered items
		○ Tax amount = 0.095 * Subtotal
		○ Total = Subtotal + Tax amount
	• Do not provide the total amount of the order until asked for. If and when asked, only mention the final sum in dollars without providing the entire calculation. As mentioned earlier, remember to be brief in your responses.
	• At the end of each conversation, always obtain the customer's name, even if provided in a single word, and attach it to the order.
	• Once the customer provides their name, confirm it and state that the order can be picked up at our cafe in 15 minutes.
	• Contoso accepts only pickup orders. Payment is accepted only at the store, and if a customer suggests paying over the phone, inform them that payments are accepted solely at the store.
Remember to always maintain a polite and professional tone.

Menu Items: 
{context}

Answer:
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(chat, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
# Returning Sources


chat_history = []
query = "what french options do you have?"
result = rag_chain.invoke({"input": query, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query), AIMessage(result['answer'])])

# returning 
for doc in result["context"]:
    print(doc)

page_content=' \n        Item Name: Pain au Chocolat\n        Pain au Chocolat details:\n            Price: 3.5$\n            Category: Bakery\n            Description: Classic French pastry filled with chocolate.\n    ' metadata={'source': 'Contoso menu from Cosmos DB'}
page_content=' \n        Item Name: Croissant\n        Croissant details:\n            Price: 2.0$\n            Category: Bakery\n            Description: Buttery, flaky pastry, freshly baked to golden perfection.\n    ' metadata={'source': 'Contoso menu from Cosmos DB'}
page_content=' \n        Item Name: Chocolate Croissant\n        Chocolate Croissant details:\n            Price: 3.0$\n            Category: Bakery\n            Description: Flaky croissant filled with rich chocolate.\n    ' metadata={'source': 'Contoso menu from Cosmos DB'}
page_content=' \n        Item Name: Cafe au Lait\n        Cafe au Lait details:\n            Price: 3.5$\n            Category: Coffees\n            Description: A coffee drink ma

In [31]:
list(set([doc.metadata['source'] for doc in result["context"]]))   

['Contoso menu from Cosmos DB']

In [19]:
import time
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
source = []
print("Welcome to Contoso Coffee Inc! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = rag_chain.invoke({"input": query, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=query), AIMessage(result['answer'])])
    end_time = time.time()
    execution_time = end_time - start_time
    print("Customer: {0}".format(query))
    print("Assistant: {0}".format(result['answer']))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")


Welcome to Contoso Coffee Inc! How can I assist you?

Customer: what french coffee options do you have?
Assistant: We offer Cafe au Lait, which is a traditional French coffee made with strong hot coffee and scalded milk. Each Cafe au Lait is priced at $3.5. Would you like to order one?
	Time taken to respond: 5 seconds
Customer: can I order 2 of them and make it hot?
Assistant: Absolutely, I have added 2 hot Cafe au Lait to your order. Do you wish to add anything else to your order?
	Time taken to respond: 6 seconds


In [20]:
chat_history

[HumanMessage(content='what french coffee options do you have?'),
 AIMessage(content='We offer Cafe au Lait, which is a traditional French coffee made with strong hot coffee and scalded milk. Each Cafe au Lait is priced at $3.5. Would you like to order one?'),
 HumanMessage(content='can I order 2 of them and make it hot?'),
 AIMessage(content='Absolutely, I have added 2 hot Cafe au Lait to your order. Do you wish to add anything else to your order?')]

In a real Q&A application we'll want some way of persisting chat history and some way of automatically inserting and updating it.

For this we can use:

- `BaseChatMessageHistory`: Store chat history. 
- `RunnableWithMessageHistory`: Lets us add message history to certain types of chains. Wrapper for an LCEL chain and a BaseChatMessageHistory that handles injecting chat history into inputs and updating it after each invocation.

In [35]:
### Statefully manage chat history ###

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


rag_chain_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [36]:
result = rag_chain_with_history.invoke(
    {"input": "What french options do you have?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(result["answer"])

Parent run 23a550d2-968f-4515-bdf7-50e66227bfc1 not found for run a1188883-bad9-4e04-bade-3840ca31e2bd. Treating as a root run.


We have several French-inspired items on our menu:

1. Pain au Chocolat ($3.5): A classic French pastry filled with chocolate.
2. Croissant ($2.0): A buttery, flaky pastry, freshly baked to golden perfection.
3. Cafe au Lait ($3.5): A coffee drink made with strong hot coffee and scalded milk. 

Please let me know which item you'd like to order or if you have any other questions.


In [37]:
result = rag_chain_with_history.invoke(
    {"input": "can I have 2 of each?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(result["answer"])

Parent run ea8aef7b-0734-4449-afc6-3ba04ec0e705 not found for run aa0b8cec-55c8-4670-a101-7ef61d0ed413. Treating as a root run.


Absolutely! Your order is as follows:

1. 2 Pain au Chocolat ($3.5 each)
2. 2 Croissants ($2.0 each)

The total for the Pain au Chocolat is $7.0 and the total for the Croissants is $4.0. Anything else you'd like to add to your order?


In [38]:
# New Session - Won't remember the previous conversation

result = rag_chain_with_history.invoke(
    {"input": "can I have 2 of each?"},
    config={"configurable": {"session_id": "def123"}},
)
print(result["answer"])


# blatantly incorrect!

Parent run 22ad147b-8f75-49c9-b876-736096852d0d not found for run ccf6b49d-b060-4660-abd7-30cdfad4e5e4. Treating as a root run.


Of course! Here's your order:

- 2 Herbal Teas (2.0$ each)
- 2 Green Teas (2.0$ each)
- 2 Chocolate Chip Cookies (2.5$ each)
- 2 Mochas (4.5$ each)
- 2 Pretzels (2.5$ each)
- 2 Cortados (3.5$ each)
- 2 Mint Teas (2.5$ each)
- 2 Chamomile Teas (2.5$ each)
- 2 Croissants (2.0$ each)
- 2 Affogatos (4.0$ each)

Please provide your name to complete the order.


In [39]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What french options do you have?'), AIMessage(content="We have several French-inspired items on our menu:\n\n1. Pain au Chocolat ($3.5): A classic French pastry filled with chocolate.\n2. Croissant ($2.0): A buttery, flaky pastry, freshly baked to golden perfection.\n3. Cafe au Lait ($3.5): A coffee drink made with strong hot coffee and scalded milk. \n\nPlease let me know which item you'd like to order or if you have any other questions."), HumanMessage(content='can I have 2 of each?'), AIMessage(content="Absolutely! Your order is as follows:\n\n1. 2 Pain au Chocolat ($3.5 each)\n2. 2 Croissants ($2.0 each)\n\nThe total for the Pain au Chocolat is $7.0 and the total for the Croissants is $4.0. Anything else you'd like to add to your order?")]),
 'def123': InMemoryChatMessageHistory(messages=[HumanMessage(content='can I have 2 of each?'), AIMessage(content="Of course! Here's your order:\n\n- 2 Herbal Teas (2.0$ each)

In [40]:
### Statefully manage chat history ###

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory
import random


def get_session_history(session_id: str) -> BaseChatMessageHistory:   
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id="shivac"
    )
    cosmos.prepare_cosmos()    
    return cosmos


rag_chain_with_cosmos_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [41]:
result = rag_chain_with_cosmos_history.invoke(
    {"input": "What french options do you have?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(result["answer"])

Parent run 0d9c7701-e372-462d-b993-93e7ef7fda76 not found for run e2ba21c0-133f-49c2-ae5a-17df42c248f5. Treating as a root run.


We have a few French-inspired items on our menu:

1. Pain au Chocolat: A classic French pastry filled with chocolate. It's priced at $3.5.
2. Croissant: A buttery, flaky pastry, freshly baked to golden perfection. It's priced at $2.0.
3. Cafe au Lait: A coffee drink made with strong hot coffee and scalded milk. It's priced at $3.5.

Please let me know if you'd like more information about these items or if you're ready to place an order.


In [42]:
result = rag_chain_with_cosmos_history.invoke(
    {"input": "can I have 2 of each?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(result["answer"])

Parent run 9f2686e9-ccfc-46d4-93c6-7626aa112287 not found for run 2c0bb5a5-dc37-425f-a6f7-e195f71d6301. Treating as a root run.


Absolutely, I've added these to your order:

1. 2x Pain au Chocolat ($3.5 each)
2. 2x Croissant ($2.0 each)
3. 2x Cafe au Lait ($3.5 each)

Is there anything else you would like to add to your order?


In [43]:
# New Session - Won't remember the previous conversation

result = rag_chain_with_cosmos_history.invoke(
    {"input": "can I have 2 of each?"},
    config={"configurable": {"session_id": "def123"}},
)
print(result["answer"])


# blatantly incorrect!

Parent run d564ee0b-ed56-4e3d-8338-3b5517fa8075 not found for run 816c2dc3-fbea-494f-bd2f-d160e5837c9f. Treating as a root run.


Sure! Here's your order:

- 2 Herbal Teas ($2.0 each)
- 2 Green Teas ($2.0 each)
- 2 Chocolate Chip Cookies ($2.5 each)
- 2 Mochas ($4.5 each)
- 2 Pretzels ($2.5 each)
- 2 Cortados ($3.5 each)
- 2 Mint Teas ($2.5 each)
- 2 Chamomile Teas ($2.5 each)
- 2 Croissants ($2.0 each)
- 2 Affogatos ($4.0 each)

Please let me know if you'd like to add, remove, or change anything in this order.


In [53]:
import time
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
source = []
print("Welcome to Contoso Coffee Inc! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = rag_chain_with_cosmos_history.invoke(
        {"input": query},
        config={"configurable": {"session_id": "abc123"}},
    )
    end_time = time.time()
    execution_time = end_time - start_time
    print("Customer: {0}".format(query))
    print("Assistant: {0}".format(result['answer']))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")

Welcome to Contoso Coffee Inc! How can I assist you?



Parent run 11842a6b-0317-4e7f-a915-1d29e7d2da29 not found for run af373d34-a1ea-4098-96e8-b7b0f58418ff. Treating as a root run.


Customer: can you make my coffees hot?
Assistant: Definitely! We'll ensure that your Cafe au Lait drinks are served hot. I've made a note of this in your order. Is there anything else you would like to specify or add to your order?
	Time taken to respond: 11 seconds


Parent run 1e8d26e6-d358-4053-9237-7b61233c888b not found for run 2e54ecfa-1b2f-4e3e-8c56-986a9e1af310. Treating as a root run.


Customer: Thanks, my name is shiva chittamuru
Assistant: Thank you, Shiva Chittamuru. Your order is confirmed and will be ready for pickup at our cafe in 15 minutes. Please make your payment at the store. Enjoy your meal!
	Time taken to respond: 10 seconds


Parent run 29f4ee0f-9bae-4c83-855d-74dffa0e202f not found for run d5b8ad3a-5155-48b2-bb20-cc1a44448c12. Treating as a root run.


Customer: thats it! thank you!
Assistant: You're welcome, Shiva Chittamuru! Thank you for choosing Contoso Coffee. We hope you enjoy your order!
	Time taken to respond: 9 seconds


In [54]:
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id="abc123",
        user_id="shivac"
    )
cosmos.prepare_cosmos()
cosmos.messages

[HumanMessage(content='What french options do you have?'),
 AIMessage(content="We have a few French-inspired items on our menu:\n\n1. Pain au Chocolat: A classic French pastry filled with chocolate. It's priced at $3.5.\n2. Croissant: A buttery, flaky pastry, freshly baked to golden perfection. It's priced at $2.0.\n3. Cafe au Lait: A coffee drink made with strong hot coffee and scalded milk. It's priced at $3.5.\n\nPlease let me know if you'd like more information about these items or if you're ready to place an order."),
 HumanMessage(content='can I have 2 of each?'),
 AIMessage(content="Absolutely, I've added these to your order:\n\n1. 2x Pain au Chocolat ($3.5 each)\n2. 2x Croissant ($2.0 each)\n3. 2x Cafe au Lait ($3.5 each)\n\nIs there anything else you would like to add to your order?"),
 HumanMessage(content='can you make my coffees hot?'),
 AIMessage(content="Definitely! We'll ensure that your Cafe au Lait drinks are served hot. I've made a note of this in your order. Is there

In [52]:
print(cosmos.session_id)
print(cosmos.user_id)

abc123
shivac
